# Anaconda3下でのInstall

condaではtensorboard実行に問題が出たので一旦これで。

```
$ pip install -i https://pypi.anaconda.org/jjhelmus/simple tensorflow
```


# Installを確認する

In [3]:
import tensorflow as tf

hello = tf.constant('Hi, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hi, TensorFlow!'


# TensorFlowを簡単な計算で理解する

> [TensorFlowを算数で理解する - Qiita](http://qiita.com/icoxfog417/items/fb5c24e35a849f8e2c5d)

### スカラ量で計算してみる

- 計算する数式
$$ 
y = x^2 + b
$$


- flowを図示すると

```
0.5 --> x--[square]--[add]--> result
                      |
3.0 --> b-------------+

```


In [11]:
def x2_plus_b(x, b):
    _x = tf.constant(x)
    _b = tf.constant(b)
    result = tf.square(_x)
    result = tf.add(result, _b)
    return result

sess = tf.Session()
print(sess.run([x2_plus_b(.5,3.)]))

[3.25]


### TensorBoardによる可視化

- 実行後に、コマンドラインから下記を実行
 
 ```
$ tensorboard --logdir=/path/to/log-directory
 ```

- `http://0.0.0.0:6006`にアクセスする

- `GRAPH`に実行済フローが複数表示される

> ポイントとしては、`tf.scalar_summary`で計算した値をsummaryしておく点です。こうして計算したsummaryを、`tf.train.SummaryWriter`で書き出していきます。 http://qiita.com/icoxfog417/items/fb5c24e35a849f8e2c5d

In [13]:
import tensorflow as tf

def monitor_calculation(x, b):
    title = "b = {0}".format(b)
    c = x2_plus_b(float(x), float(b))
    s = tf.scalar_summary(title, c)
    m = tf.merge_summary([s])  # if you are using some summaries, merge them
    return m

with tf.Session() as sess:
    writer = tf.train.SummaryWriter("log", graph_def=sess.graph_def)    
    xaxis = range(-10, 12)

    for b in range(3):
        for x in xaxis:
            summary_str = sess.run(monitor_calculation(x, b))
            writer.add_summary(summary_str, x)

### スカラ量で計算する

> https://www.tensorflow.org/versions/master/get_started/index.html

上記コードはPython2ベースのため、`xrange()` -> `range()`に修正。

> 参考 http://stackoverflow.com/questions/17192158/nameerror-global-name-xrange-is-not-defined-in-python-3

- やっていることは回帰分析

> 「y = 0.1 x + 0.3」という数式を使って生成したxとyを学習データとして、y = W x + b という数式のWとbを最適化していく http://qiita.com/MATS_ELB/items/dfc50149d52e47e5a07b


In [20]:
import tensorflow as tf
import numpy as np

# Create 100 phony x, y data points in NumPy, y = x * 0.1 + 0.3
x_data = np.random.rand(100).astype(np.float32)
y_data = x_data * 0.1 + 0.3

# Try to find values for W and b that compute y_data = W * x_data + b
# (We know that W should be 0.1 and b 0.3, but Tensorflow will
# figure that out for us.)
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))
y = W * x_data + b

# Minimize the mean squared errors.
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# Before starting, initialize the variables.  We will 'run' this first.
init = tf.initialize_all_variables()

# Launch the graph.
sess = tf.Session()
sess.run(init)

# Fit the line.
for step in range(201):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))

# Learns best fit is W: [0.1], b: [0.3]

0 [-0.07575795] [ 0.5411855]
20 [ 0.02578249] [ 0.33941314]
40 [ 0.07675866] [ 0.31234229]
60 [ 0.09272193] [ 0.30386502]
80 [ 0.09772088] [ 0.30121034]
100 [ 0.09928628] [ 0.30037904]
120 [ 0.0997765] [ 0.30011871]
140 [ 0.09993] [ 0.30003718]
160 [ 0.09997807] [ 0.30001166]
180 [ 0.09999313] [ 0.30000365]
200 [ 0.09999785] [ 0.30000114]


### テンソルで計算する

> http://qiita.com/MATS_ELB/items/fec7f54de2dd18b043ae

In [28]:
# tfという名前で参照できるようにtensorflowを、npという名前で参照できるようにインポート
import tensorflow as tf
import numpy as np

# y_data = W_data * x_data + b_data というx_dataとy_dataの関係になるようにあらかじめ
# W_dataとb_dataの値を定める。機械学習が適切に行われるとWはこのW_dataに、bはこのb_dataに近づく
# なお、W_dataは２行２列のテンソル、b_dataは２行１列のテンソル
W_data = np.array([[0.1, 0], [0, 0.1]])
b_data = np.array([0.3, 0.3])

# 乱数生成を利用して0から1の間の数値を持つ２行１列の行列「X_data」を浮動小数として100個生成
x_data = np.random.rand(100, 2, 1).astype("float32")

# その後で、生成した100個のxに対して、行列版でのy=0.1x+0.3となるようなyを100個生成
y_data = W_data * x_data + b_data

# 上記で生成したxとy（共に２行１列のテンソル）の組を学習データとして用いる

# 機械学習で最適化するWとbを設定する。Wは２行２列のテンソル。bは２行１列のテンソル。
W = tf.Variable(tf.random_uniform([2, 2], -1.0, 1.0))
b = tf.Variable(tf.zeros([2]))
y = W * x_data + b

# 学習において、その時点での学習のダメ程度を表すlossを、学習データのyとその時点でのyの差の２乗と定義
# Wとbの最適化のアルゴリズムを最急降下法（勾配法）とし、その１回の最適化処理にoptimizerと名前を付ける
# 上記の最適化処理の繰り返しによりlossを最小化する処理をtrainと呼ぶことにする
loss = tf.reduce_mean(tf.square(y_data - y))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# 学習を始める前にこのプログラムで使っている変数を全てリセットして空っぽにする
init = tf.initialize_all_variables()

# Launch the graph.（おきまりの文句）
sess = tf.Session()
sess.run(init)

# 学習を1000回行い、100回目ごとに画面に学習回数とWとbのその時点の値を表示する
for step in range(1001):
    sess.run(train)
    if step % 100 == 0:
        print(step, sess.run(W), sess.run(b))

# Learns best fit is W: [[0.1, 0], [0, 0.1]], b: [0.3, 0.3]

0 [[ 0.12049839  0.63580847]
 [ 0.7111249  -0.02813603]] [ 0.05483134  0.09086838]
100 [[ 0.15628858  0.04232659]
 [ 0.05392477  0.14043736]] [ 0.27025187  0.27766153]
200 [[ 0.10838222  0.00629473]
 [ 0.00802105  0.10602347]] [ 0.29557258  0.29667521]
300 [[ 0.10124753  0.00093688]
 [ 0.00119379  0.10089649]] [ 0.29934105  0.29950514]
400 [[ 0.10018568  0.00013946]
 [ 0.0001777   0.10013343]] [ 0.29990193  0.29992634]
500 [[  1.00027628e-01   2.07803114e-05]
 [  2.64537048e-05   1.00019872e-01]] [ 0.29998541  0.29998901]
600 [[  1.00004129e-01   3.12404495e-06]
 [  3.96192627e-06   1.00002974e-01]] [ 0.29999781  0.29999834]
700 [[  1.00000627e-01   5.01336388e-07]
 [  6.14700070e-07   1.00000471e-01]] [ 0.29999965  0.29999974]
800 [[  1.00000247e-01   2.22809277e-07]
 [  2.17059082e-07   1.00000232e-01]] [ 0.29999986  0.29999986]
900 [[  1.00000247e-01   2.22755844e-07]
 [  2.14493255e-07   1.00000232e-01]] [ 0.29999986  0.29999986]
1000 [[  1.00000247e-01   2.22767198e-07]
 [  2.1447

### 次は同内容のTensorboard可視化（エラーあり）

> http://qiita.com/MATS_ELB/items/fec7f54de2dd18b043ae

In [ ]:
# tfという名前で参照できるようにtensorflowを、npという名前で参照できるようにインポート
import tensorflow as tf
import numpy as np

# y_data = W_data * x_data + b_data というx_dataとy_dataの関係になるようにあらかじめ
# W_dataとb_dataの値を定める。機械学習が適切に行われるとWはこのW_dataに、bはこのb_dataに近づく
# なお、W_dataは２行２列のテンソル、b_dataは２行１列のテンソル
W_data = np.array([[0.1, 0], [0, 0.1]])
b_data = np.array([0.3, 0.3])

# 乱数生成を利用して0から1の間の数値を持つ２行１列の行列「X_data」を浮動小数として100個生成
x_data = np.random.rand(100, 2, 1).astype("float32")

# その後で、生成した100個のxに対して、行列版でのy=0.1x+0.3となるようなyを100個生成
y_data = W_data * x_data + b_data

# 上記で生成したxとy（共に２行１列のテンソル）の組を学習データとして用いる

# 機械学習で最適化するWとbを設定する。Wは２行２列のテンソル。bは２行１列のテンソル。
W = tf.Variable(tf.random_uniform([2, 2], -1.0, 1.0))
b = tf.Variable(tf.zeros([2]))
y = W * x_data + b

## TensorBoardへ表示するための変数を用意する（ヒストグラム用）
W_hist = tf.histogram_summary("weights", W)
b_hist = tf.histogram_summary("biases", b)
y_hist = tf.histogram_summary("y", y)

# 学習において、その時点での学習のダメ程度を表すlossを、学習データのyとその時点でのyの差の２乗と定義
# Wとbの最適化のアルゴリズムを最急降下法（勾配法）とし、その１回の最適化処理にoptimizerと名前を付ける
# 上記の最適化処理の繰り返しによりlossを最小化する処理をtrainと呼ぶことにする
loss = tf.reduce_mean(tf.square(y_data - y))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

## TensorBoardへloss（学習のダメ具合の指標として設定したスカラー値）を表示するための変数を用意する（イベント用）
loss_sum = tf.scalar_summary("loss", loss)

sess = tf.Session()

# 上記で用意した合計４つのTensorBoard描画用の変数を、TensorBoardが利用するSummaryデータとしてmerge（合体）する
# また、そのSummaryデータを書き込むSummaryWriterを用意し、書き込み先を'/tmp/tf_logs'ディレクトリに指定する
merged = tf.merge_all_summaries()
writer = tf.train.SummaryWriter("/tmp/tf_logs", sess.graph_def)


# 学習を始める前にこのプログラムで使っている変数を全てリセットして空っぽにする
init = tf.initialize_all_variables()

# Launch the graph.（おきまりの文句）
sess.run(init)

# 学習を1000回行い、10回目ごとにSummaryデータを保存し、画面にWとbのその時点の値を表示する
for step in range(1001):
    if step % 10 == 0:
        result = sess.run([merged, loss])
        summary_str = result[0]
        writer.add_summary(summary_str, step)
        print(step, sess.run(W), sess.run(b))
    else:
        sess.run(train)

# Learns best fit is W: [[0.1, 0], [0, 0.1]], b: [0.3, 0.3]

# TenorFlow Tutorials

### MNIST for ML beginners (91%)

> [TensorFlow::MNIST for ML beginners](https://www.tensorflow.org/versions/master/tutorials/mnist/beginners/index.html#mnist-for-ml-beginners)

In [29]:
# Implementing the Regression
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

# Training
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_:batch_ys})

# Evaluating Our Model
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0.9166


### Deep MNIST for Experts (99%)

> https://www.tensorflow.org/versions/master/tutorials/mnist/pros/index.html#deep-mnist-for-experts

- 翻訳

> http://qiita.com/KojiOhki/items/64a2ee54214b01a411c7

- ソースは下記より(macbook pro early2015 - 22 sec /100 steps -> 73 min / 20000 steps)

> http://qiita.com/haminiku/items/36982ae65a770565458d

In [ ]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import, unicode_literals
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# mnistデータ読み込み
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

##########################################
# 深層畳み込みニューラルネットワークを構築
# Build a Multilayer Convolutional Network
# 精度91%は悪いから深層畳み込みモデルを構築して99.2%を目指す
###########################################

"""
ちょっと理解できませんでした.. 
多層になると損失関数のパラメータ勾配が限りなくゼロに近づく勾配消失問題(Vanishing gradient problem)対策のために、少量のノイズで重みを初期化する関数みたいです。

Weight Initialization

To create this model, we're going to need to create a lot of weights and biases.
One should generally initialize weights with a small amount of noise for symmetry breaking,
and to prevent 0 gradients. Since we're using ReLU neurons, it is also good practice to initialize
them with a slightly positive initial bias to avoid "dead neurons." Instead of doing this repeatedly
while we build the model, let's create two handy functions to do it for us.
"""


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

"""
Convolution and Pooling
TensorFlow also gives us a lot of flexibility in convolution and pooling operations.
How do we handle the boundaries? What is our stride size? In this example,
we're always going to choose the vanilla version. Our convolutions uses a stride of one
and are zero padded so that the output is the same size as the input. Our pooling is plain old
max pooling over 2x2 blocks. To keep our code cleaner, let's also abstract those operations into functions.
"""


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')
"""
第1レイヤー 5x5パッチで32の特徴を計算
[5, 5, 1, 32] は最初の5,5はパッチサイズ,1は入力チャンネル数,32は出力チャンネル数
"""
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

"""
第2レイヤー 5x5パッチで64の特徴を計算
"""
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

"""
密集接続レイヤー

画像サイズ7x7に還元されているので、1024のニューロンと完全に接続する層（翻訳がかなり怪しいので原文読んでください）MNISTデータは28x28ピクセルなので1/16ずつ読むみたいです。

Densely Connected Layer

Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons to allow
processing on the entire image. We reshape the tensor from the pooling layer into a batch of vectors,
multiply by a weight matrix, add a bias, and apply a ReLU.
"""
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

"""
過剰適合を排除する

Dropout

To reduce overfitting, we will apply dropout before the readout layer. We create a placeholder
for the probability that a neuron's output is kept during dropout. This allows us to turn dropout
on during training, and turn it off during testing. TensorFlow's tf.nn.dropout op automatically
handles scaling neuron outputs in addition to masking them, so dropout just works without any additional scaling.
"""
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

"""
読み出し層
第1層のロジスティック回帰のように、ロジスティック回帰層を追加

Readout Layer
Finally, we add a softmax layer, just like for the one layer softmax regression above.
"""
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

"""
モデルの学習と評価
TensorFlowを使用して、洗練された深い学習モデルの学習と評価を行います。
"""
cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.initialize_all_variables())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g" % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

# 結果表示
print("test accuracy %g" % accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

# TensorFlow公式レポジトリを使う

### git cloneする

```
$ git clone https://github.com/tensorflow/tensorflow
```

`/tensorflow/tensorflow/examples`にtutorials等のsource codeあり。
